In [1]:
import findspark
findspark.init('/home/shenjeffrey/spark/')
import pyspark
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns

In [3]:
# initiate spark
sc = pyspark.SparkContext()

In [4]:
# Read in data
data = sc.textFile("shakespeare.txt")
data.take(10)

[u'1609',
 u'',
 u'THE SONNETS',
 u'',
 u'by William Shakespeare',
 u'',
 u'',
 u'',
 u'                     1',
 u'  From fairest creatures we desire increase,']

In [5]:
data = data.flatMap(lambda x: x.split(" "))

In [6]:
data.take(10)

[u'1609',
 u'',
 u'THE',
 u'SONNETS',
 u'',
 u'by',
 u'William',
 u'Shakespeare',
 u'',
 u'']

In [7]:
test = 'ADH.'
test2 = '234'

In [8]:
test.isdigit()

False

In [9]:
test2.isdigit()

True

In [10]:
test.upper()

'ADH.'

In [11]:
test.upper() == test

True

In [12]:
def filterNonWords(word):
    # take out empty strings
    if word == '':
        return False
    # take out words w/ only numbers
    elif word.isdigit():
        return False
    # take out words with all Capital characters
    # will also take out words with Capital characters w/ a period at the end
    elif word == word.upper():
        return False
    else:
        return True

In [13]:
clean_data = data.filter(filterNonWords)

In [14]:
clean_data.count()

818620

In [15]:
clean_data.take(20)

[u'by',
 u'William',
 u'Shakespeare',
 u'From',
 u'fairest',
 u'creatures',
 u'we',
 u'desire',
 u'increase,',
 u'That',
 u'thereby',
 u"beauty's",
 u'rose',
 u'might',
 u'never',
 u'die,',
 u'But',
 u'as',
 u'the',
 u'riper']

In [16]:
clean_data2 = clean_data.collect()[1:] + [""]
clean_data3 = clean_data2[1:] + [""]

In [17]:
# Put them back into RDD
clean_data2 = sc.parallelize(clean_data2)
clean_data3 = sc.parallelize(clean_data3)

In [18]:
print clean_data.take(10)
print clean_data2.take(10)
print clean_data3.take(10)

[u'by', u'William', u'Shakespeare', u'From', u'fairest', u'creatures', u'we', u'desire', u'increase,', u'That']
[u'William', u'Shakespeare', u'From', u'fairest', u'creatures', u'we', u'desire', u'increase,', u'That', u'thereby']
[u'Shakespeare', u'From', u'fairest', u'creatures', u'we', u'desire', u'increase,', u'That', u'thereby', u"beauty's"]


In [19]:
clean_data = clean_data.zipWithIndex().map(lambda (x,y): (y,x))
clean_data2 = clean_data2.zipWithIndex().map(lambda (x,y): (y,x))
clean_data3 = clean_data3.zipWithIndex().map(lambda (x,y): (y,x))

In [20]:
print clean_data.take(10)
print clean_data2.take(10)
print clean_data3.take(10)

[(0, u'by'), (1, u'William'), (2, u'Shakespeare'), (3, u'From'), (4, u'fairest'), (5, u'creatures'), (6, u'we'), (7, u'desire'), (8, u'increase,'), (9, u'That')]
[(0, u'William'), (1, u'Shakespeare'), (2, u'From'), (3, u'fairest'), (4, u'creatures'), (5, u'we'), (6, u'desire'), (7, u'increase,'), (8, u'That'), (9, u'thereby')]
[(0, u'Shakespeare'), (1, u'From'), (2, u'fairest'), (3, u'creatures'), (4, u'we'), (5, u'desire'), (6, u'increase,'), (7, u'That'), (8, u'thereby'), (9, u"beauty's")]


In [21]:
n_gram = clean_data.join(clean_data2).join(clean_data3)
n_gram.take(10)

[(0, ((u'by', u'William'), u'Shakespeare')),
 (655360, ((u'dagger.]', u'This'), u'is')),
 (524290, ((u'most', u'of'), u'me!')),
 (393220, ((u'in', u'a'), u'country')),
 (262150, ((u'here', u'in'), u'arms')),
 (786440, ((u'sweet', u'consort;'), u'to')),
 (655370, ((u'die.', u'She'), u'stabs')),
 (524300, ((u'and', u'might'), u'To')),
 (393230, ((u'and', u'Burgundy;'), u'Attendants.')),
 (262160, ((u"King's,", u'we'), u'charge'))]

In [22]:
n_gram = n_gram.map(lambda (key, val): ((val[0][0], val[0][1],val[1]), (1)))
n_gram.take(10)

[((u'by', u'William', u'Shakespeare'), 1),
 ((u'dagger.]', u'This', u'is'), 1),
 ((u'most', u'of', u'me!'), 1),
 ((u'in', u'a', u'country'), 1),
 ((u'here', u'in', u'arms'), 1),
 ((u'sweet', u'consort;', u'to'), 1),
 ((u'die.', u'She', u'stabs'), 1),
 ((u'and', u'might', u'To'), 1),
 ((u'and', u'Burgundy;', u'Attendants.'), 1),
 ((u"King's,", u'we', u'charge'), 1)]

In [23]:
n_gram = n_gram.reduceByKey(lambda x,y: x+y)
n_gram.take(10)

[((u'we', u'do', u'establish'), 1),
 ((u'world.', u'You', u'are'), 1),
 ((u'not', u'permanent-', u'sweet,'), 1),
 ((u'a', u'friend', u'of'), 8),
 ((u'whale', u'on', u'ground,'), 1),
 ((u'us', u'consult', u'upon'), 1),
 ((u'The', u'breath', u'no'), 1),
 ((u'speak', u'What', u'have'), 1),
 ((u'radiant', u'Queen', u'hates'), 1),
 ((u'he,', u'nor', u'his'), 1)]

In [24]:
n_gram = n_gram.map(lambda (key, val): ((key[0],key[1]),(key[2], val)))
n_gram.take(10)

[((u'we', u'do'), (u'establish', 1)),
 ((u'world.', u'You'), (u'are', 1)),
 ((u'not', u'permanent-'), (u'sweet,', 1)),
 ((u'a', u'friend'), (u'of', 8)),
 ((u'whale', u'on'), (u'ground,', 1)),
 ((u'us', u'consult'), (u'upon', 1)),
 ((u'The', u'breath'), (u'no', 1)),
 ((u'speak', u'What'), (u'have', 1)),
 ((u'radiant', u'Queen'), (u'hates', 1)),
 ((u'he,', u'nor'), (u'his', 1))]

In [25]:
n_gram = n_gram.groupByKey().mapValues(list)
n_gram.take(10)

[((u'shadows;', u'and'), [(u'the', 1)]),
 ((u'meantime,', u'lady,'), [(u"I'll", 1)]),
 ((u'Where', u'intend'), [(u'holy', 1)]),
 ((u'Exit', u'Hast'), [(u'thou', 1)]),
 ((u'fools', u'have'), [(u'brought', 1), (u'was', 1)]),
 ((u'blastments', u'are'), [(u'most', 1)]),
 ((u'stretching', u'thee,'), [(u'And', 1)]),
 ((u"'twere", u'far'), [(u'off;', 1)]),
 ((u'sight', u'For'), [(u'terror,', 1), (u'fear', 1)]),
 ((u'Play,', u'music,'), [(u'then.', 1)])]

In [26]:
result = n_gram.collectAsMap()


In [27]:
result['Now', 'is']

[(u'the', 9),
 (u'he', 1),
 (u'it', 3),
 (u'this', 1),
 (u'be', 1),
 (u'his', 1),
 (u'your', 1),
 (u'a', 1),
 (u'that', 1),
 (u'my', 2),
 (u'Mortimer', 1)]

In [101]:
def generate_weights(third_word_list):
    weights = []
    for word in third_word_list:
#         print word
        weights += [word[0]] * word[1]
    return weights
        

In [138]:
def generate_sentence(n_gram, sentence_length=20):
    # Declare blank sentence
    sentence = []
    # Take a random sample from the RDD
#     random_sample = n_gram.takeSample(True, 1,12)
    random_sample = n_gram.takeSample(True, 1)
    # First word
    first_word = random_sample[0][0][0]
    second_word = random_sample[0][0][1]
    third_word_list = random_sample[0][1]
    look_up_key = (first_word, second_word)
#     print "first word: ",first_word
#     print "second word: ",second_word
#     print "third word list: ",third_word_list
#     print "starting lookup: ",look_up_key
    sentence += [first_word] + [second_word]
#     print "start sentence: ",sentence
    # loop until sentence length is greater than 20
    while len(sentence) < sentence_length:
        # create weights
        weights = generate_weights(third_word_list)
#         print "weights: ", weights
        # use np.random.choice
        third_word = np.random.choice(weights)
#         print "third word: ", third_word
        
        # update sentence and look up key
        sentence += [third_word]
        look_up_key = (look_up_key[1], third_word)
#         print "look_up_key: ", look_up_key
        third_word_list = n_gram.map(lambda x: x).lookup(look_up_key)
#         print "new third word list: ", third_word_list
        third_word_list = third_word_list[0]
        
    print ' '.join(sentence)
    return ' '.join(sentence)
    

In [140]:
for i in xrange(10):
    generate_sentence(n_gram, 20)

and, not consulting, broke Into a thousand grains That issue out of hearing To th' vulgar eye, that false peer,
chokes The hollow passage of their particular functions and wonder if Titania be awak'd; Then, what it is. Come, go
Than all the hot-bloods between fourteen and fourteen and a little before. [Exeunt all but would fain have drink. This
Bohemia can learnedly handle, though they are no venereal signs. Vengeance is in GIoucestershire- 'Twas where you are, Make them
Count Did see her, hear her, at that instant Crav'd audience; and wherein It shall be while you are waspish.
Princes. Exit London. street Enter and at a strife? What is the day appointed for the witch of Brainford. But
Montano, you were wont. There's a good husband. Pedro. And when have requir'd Some heavenly power guide us Out of
have outfac'd them all. Let it stamp wrinkles in my life, by some other messenger. Back, slave, or thou look'st
For Doll is in. Pistol speaks nought but fame; And every tongue that speaks thu